In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import toolkitETL
import importlib
importlib.reload(toolkitETL)

<module 'toolkitEngineering' from 'd:\\Projetos\\ETL-ML-Dengue\\src\\engineering\\toolkitEngineering.py'>

In [2]:
## Estabelecendo diretórios base
diretorio_src_etl = os.getcwd()
diretorio_src = Path(diretorio_src_etl).parent.absolute()
diretorio_base = Path(diretorio_src).parent.absolute()
diretorio_data = os.path.join(diretorio_base, 'data')

# Bronze
diretorio_bronze = os.path.join(diretorio_data, 'bronze')
diretorio_cnes_st_bronze = os.path.join(diretorio_bronze, 'cnes_st')
diretorio_cnes_st_bronze_csv = os.path.join(diretorio_cnes_st_bronze, 'csv')

# Silver
diretorio_silver = os.path.join(diretorio_data, 'silver')
diretorio_cnes_st_silver = os.path.join(diretorio_silver, 'cnes_st')
diretorio_cnes_st_silver_tsv = os.path.join(diretorio_cnes_st_silver, 'tsv')

## Criação de pastas para armazenar os arquivos
os.makedirs(os.path.join(diretorio_data), exist_ok=True)
os.makedirs(os.path.join(diretorio_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_cnes_st_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_cnes_st_silver_tsv), exist_ok=True)




In [9]:
arquivos_csv = [arquivo for arquivo in os.listdir(diretorio_cnes_st_bronze_csv) if arquivo.endswith("csv")]
arquivos_csv = arquivos_csv[-5:]
arquivos_tsv = [arquivo.rstrip("csv") + "tsv" for arquivo in arquivos_csv]
arquivos_tsv

['2010_1_2010_12_AC_CNES-ST.tsv', '2020_1_2020_12_AC_CNES-ST.tsv']

In [7]:
colunas_importantes = ['CNES', 'CODUFMUN', 'COMPETEN', 'VINC_SUS', 'PF_PJ', 'NIV_DEP', 'TPGESTAO',
                       'ORGEXPED', 'AV_ACRED', 'CLASAVAL', 'AV_PNASS', 'GESPRG1E', 'GESPRG1M',
                       'GESPRG2E', 'GESPRG2M', 'GESPRG4E', 'GESPRG4M', 'NIVATE_A', 'GESPRG3E',
                       'GESPRG3M', 'GESPRG5E', 'GESPRG5M', 'GESPRG6E', 'GESPRG6M', 'NIVATE_H',
                       'QTLEITP1', 'QTLEITP2', 'QTLEITP3', 'LEITHOSP', 'QTINST01', 'QTINST02',
                       'QTINST03', 'QTINST04', 'QTINST05', 'QTINST06', 'QTINST07', 'QTINST08', 
                       'QTINST09', 'QTINST10', 'QTINST11', 'QTINST12', 'QTINST13', 'QTINST14',
                       'URGEMERG', "QTINST15", "QTINST16", "QTINST17", "QTINST18", "QTINST19", 
                       "QTINST20", "QTINST21", "QTINST22", "QTINST23", "QTINST24", "QTINST25", 
                       "QTINST26", "QTINST27", "QTINST28", "QTINST29", "QTINST30", "ATENDAMB",
                       "QTINST31", "QTINST32", "QTINST33", "CENTRCIR", "QTINST34", "QTINST35", 
                       "QTINST36", "QTINST37", "CENTROBS", "QTLEIT05", "QTLEIT06", "QTLEIT07", 
                       "QTLEIT08", "QTLEIT09", "QTLEIT19", "QTLEIT20", "QTLEIT21", "QTLEIT22", 
                       "QTLEIT23", "QTLEIT32", "QTLEIT34", "QTLEIT38", "QTLEIT39", "QTLEIT40",
                       "CENTRNEO", "ATENDHOS", "SERAP01P", "SERAP01T", "SERAP02P", "SERAP02T", 
                       "SERAP03P", "SERAP03T", "SERAP04P", "SERAP04T", "SERAP05P", "SERAP05T", 
                       "SERAP06P", "SERAP06T", "SERAP07P", "SERAP07T", "SERAP08P", "SERAP08T", 
                       "SERAP09P", "SERAP09T", "SERAP10P", "SERAP10T", "SERAP11P", "SERAP11T", 
                       "SERAPOIO", "RES_BIOL", "RES_QUIM", "RES_RADI", "RES_COMU", "COLETRES", 
                       "COMISS01", "COMISS02", "COMISS03", "COMISS04", "COMISS05", "COMISS06", 
                       "COMISS07", "COMISS08", "COMISS09", "COMISS10", "COMISS11", "COMISS12", 
                       "COMISSAO", "AP01CV01", "AP01CV02", "AP01CV03", "AP01CV04", "AP01CV05", 
                       "AP01CV06", "AP02CV01", "AP02CV02", "AP02CV03", "AP02CV04", "AP02CV05", 
                       "AP02CV06", "AP03CV01", "AP03CV02", "AP03CV03", "AP03CV04", "AP03CV05", 
                       "AP03CV06", "AP04CV01", "AP04CV02", "AP04CV03", "AP04CV04", "AP04CV05", 
                       "AP04CV06", "AP05CV01", "AP05CV02", "AP05CV03", "AP05CV04", "AP05CV05", 
                       "AP05CV06", "AP06CV01", "AP06CV02", "AP06CV03", "AP06CV04", "AP06CV05", 
                       "AP06CV06", "AP07CV01", "AP07CV02", "AP07CV03", "AP07CV04", "AP07CV05", 
                       "AP07CV06", "ATEND_PR"




]

colunas_decodificar = {
    'GESPRG1E':
    {1: "Sim",
    0: "Não"},
}

column_map = []

columns_to_convert = []

In [8]:
for arquivo_csv, arquivo_tsv in zip(arquivos_csv, arquivos_tsv):
    df = pd.read_csv(os.path.join(diretorio_cnes_st_bronze_csv, arquivo_csv), sep=',')

    for coluna in colunas_importantes:
        if coluna not in df.columns:
            df[coluna] = np.nan
            continue
    df = df[colunas_importantes]

    for coluna in colunas_decodificar:
        df[coluna] = df[coluna].apply(lambda x: toolkitETL.aplicar_decoficador(x, coluna, colunas_decodificar))
    df.replace("NA", np.nan, inplace=True)
    df.replace("", np.nan, inplace=True)
    df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
    df['NU_MES'] = df['COMPETEN'].apply(lambda x: str(x)[-2:].lstrip("0"))
    df['NU_ANO'] = df['COMPETEN'].apply(lambda x: str(x)[:4])
    # df = df.rename(columns=column_map)
    
    df.columns = df.columns.str.lower()
    
    # for column in columns_to_convert:
    #     df[column] = pd.to_numeric(df[column], errors='coerce').astype('Int64')

    df.to_csv(os.path.join(diretorio_cnes_st_silver_tsv, arquivo_tsv), sep='\t', index=False)

In [14]:
diretorio_dotenv = os.path.join(diretorio_base, ".env")

cursor, conn = toolkitETL.conectar_banco(diretorio_dotenv)
for arquivo_tsv in arquivos_tsv:
    
    caminho_arquivo = os.path.join(diretorio_cnes_st_silver_tsv, arquivo_tsv)
    toolkitETL.inserir_no_banco(cursor, conn, caminho_arquivo, 'cnes_st', delimiter='\t',
                                        diretorio_dotenv=diretorio_dotenv)
